In [ ]:
%pip install sklearn
%pip transformers
%pip install huggingface
%pip install torch 
%pip install pandas
%pip install sklearn|
%pip install datasets
%pip install transformers
%pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: unknown command "transformers"
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: -c: line 1: syntax error: unexpected end of file
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.9 MB/s 
     |████████████████████████████████| 212 kB 85.9 MB/s 
     |████████████████████████████████| 141 kB 85.9 MB/s 
     |████████████████████████████████| 101 kB 13.7 MB/s 
     |████████████████████████████████| 596 kB 85.3 MB/s 
     |████████████████████████████████| 127 kB 80.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installat

In [ ]:

from datasets import load_dataset, load_metric
#from sklearn.datasets import fetch_20newsgroups #encountered issues SSLCertificationError with this 
from datasets import load_dataset
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

from spacy import tokenizer
from spacy.lang.en import English
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer,AdamW,get_scheduler


In [ ]:
f1_score = load_metric("f1")

def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return f1_score.compute(predictions = predictions, references = label)

def tokenize(batch):
  return longformer_tokenizer(batch['text'])

In [ ]:
hyperpartisan_dataset = load_dataset("hyperpartisan_news_detection", "bypublisher", "/data") #Not stripped of headers and footers
hyperpartisan_dataset.keys()

Using custom data configuration bypublisher-data_dir=%2Fdata


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/600000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/600000 [00:00<?, ? examples/s]

Dataset hyperpartisan_news_detection downloaded and prepared to /root/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher-data_dir=%2Fdata/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

dict_keys(['train', 'validation'])

In [ ]:
import json

In [ ]:
distribution_name = "hyperpartisan_distribution"

with open(distribution_name, 'r') as ins:
    distribution= json.load(ins)

seed = distribution['seed']

train_index = distribution['train']
eval_index = distribution['eval']
test_index = distribution['test']

In [ ]:
train_index_for_longformers = train_index['Short'] + train_index["BERT"] + train_index['Long']
eval_index_for_longformers = eval_index['Short'] + eval_index["BERT"] + eval_index['Long']
test_index_for_longformers = test_index['Short'] + test_index["BERT"] + test_index['Long']

In [ ]:
#change to bypublisher because byarticle has no test set.
hyperpartisan_train = hyperpartisan_dataset['train']
hyperpartisan_valid = hyperpartisan_dataset['validation']

hyperpartisan_train = hyperpartisan_train.shuffle(seed = 32)
hyperpartisan_valid = hyperpartisan_valid.shuffle(seed = 32)

In [ ]:
#Split validation data into test and train_set
eval_test_split = hyperpartisan_valid.train_test_split(test_size = 0.5)

hyperpartisan_valid = eval_test_split['train']
hyperpartisan_test = eval_test_split['test']

In [ ]:
int_labels_train = [1 if x == True else 0 for x in hyperpartisan_train['hyperpartisan']]
int_labels_valid = [1 if x == True else 0 for x in hyperpartisan_valid['hyperpartisan']]
int_labels_test = [1 if x == True else 0 for x in hyperpartisan_test['hyperpartisan']]

In [ ]:
hyperpartisan_train = hyperpartisan_train.add_column("labels", int_labels_train)
hyperpartisan_valid= hyperpartisan_valid.add_column("labels", int_labels_valid)
hyperpartisan_test = hyperpartisan_test.add_column("labels", int_labels_test)

Flattening the indices:   0%|          | 0/600 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/300 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/300 [00:00<?, ?ba/s]

In [ ]:
hyperpartisan_train = hyperpartisan_train.remove_columns(["title","hyperpartisan","url","published_at","bias"])
hyperpartisan_valid = hyperpartisan_valid.remove_columns(["title","hyperpartisan","url","published_at","bias"])
hyperpartisan_test = hyperpartisan_test.remove_columns(["title","hyperpartisan","url","published_at","bias"])

In [ ]:
hyperpartisan_train = hyperpartisan_train.select(train_index_for_longformers)
hyperpartisan_valid = hyperpartisan_valid.select(eval_index_for_longformers)
hyperpartisan_test = hyperpartisan_test.select(test_index_for_longformers)

In [ ]:
#extract long documents from test and eval
#evaluate on test and eval
#Use test_json to have index of long_files and short files and normal files as well as labels

In [ ]:
hyperpartisan_train = hyperpartisan_train.select(range(1000))
hyperpartisan_valid = hyperpartisan_valid.select(range(1000))
hyperpartisan_test = hyperpartisan_test.select(range(1000))

In [ ]:
hyperpartisan_train = hyperpartisan_train.map(tokenize)
hyperpartisan_valid = hyperpartisan_valid.map(tokenize)
hyperpartisan_test = hyperpartisan_test.map(tokenize)

  0%|          | 0/588846 [00:00<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (5075 > 4096). Running this sequence through the model will result in indexing errors


In [ ]:
hyperpartisan_train = hyperpartisan_train.remove_columns(['text'])
hyperpartisan_valid = hyperpartisan_valid.remove_columns(['text'])
hyperpartisan_test = hyperpartisan_test.remove_columns(['text'])

In [ ]:
input.to_json("longformer_input")

Creating json from Arrow format:   0%|          | 0/59 [00:00<?, ?ba/s]

4012419622

In [ ]:
#FINETUNING TO A NUMBER OF Labels
from transformers import LongformerTokenizer, LongformerForSequenceClassification,LongformerConfig

longformer_tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

longformer_model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                  gradient_checkpointing=False,
                                                  attention_window = 512,
                                                  num_labels = 2,
                                                  return_dict=True)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', '

In [ ]:
import torch.quantization
model = torch.quantization.quantize_dynamic(longformer_model, {torch.nn.Linear}, dtype=torch.qint8)

#model = model.to(device)

In [ ]:
with torch.no_grad():
  input = longformer_tokenizer(hyperpartisan_train['text'], return_tensors= "pt")
  out = longformer_model(**input).logits

  labels = out.argmax().item()

In [ ]:
labels, hyperpartisan_train['hyperpartisan']

FINETUNING

In [ ]:
import optuna

In [ ]:
def hp_space(trial):
  return {
          "num_train_epochs":trial.suggest_int("num_training_epochs",1,5),
          "learning_rate": trial.suggest_float("learning_rate",1e-5,2e-5),
  }

In [ ]:
torch.device("cuda")

device(type='cuda')

In [ ]:

def longformer_init():
  return (LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                  gradient_checkpointing=False,
                                                  attention_window = 512,
                                                  num_labels = 2,
                                                  return_dict=True))
                                                                     
data_collator = DataCollatorWithPadding(tokenizer=longformer_tokenizer) #to pad sequences

#default training arguments 
training_args = TrainingArguments(output_dir="./results", per_device_train_batch_size=8, 
                                  evaluation_strategy= "epoch", per_device_eval_batch_size=8, num_train_epochs=5)

#fine-tuning model
longformer_finetuned = Trainer(model_init = longformer_init, 
              model = longformer_model,
              args = training_args,
              train_dataset= hyperpartisan_train,
              eval_dataset= hyperpartisan_valid,
              tokenizer= longformer_tokenizer,
              data_collator=data_collator,
              compute_metrics= compute_metrics)

#searching for best hyperparameters
best_run = longformer_finetuned.hyperparameter_search(direction = "maximize", hp_space = hp_space)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:350: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  FutureWarning,
[I 2022-07-29 12:17:41,434] A new study created in memory with name: no-name-a957995d-965f-4a75-a306-c0287e68d832
Trial:
loading configuration file https://huggingface.co/allenai/longformer-base-4096/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0690955d8f70934f95adf0fb108d5f7322d02f8d7dd938b7b133cb7421e120e6.b25f41ff6acdcb7ab47c505c70e351b3fc01957b3798197e5ac6e8efc547ac

In [ ]:

import gc
del eval_index
del eval_index_for_longformers
gc.collect()


211

In [ ]:
torch.cuda.m

0

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=longformer_tokenizer) #to pad sequences

#default training arguments 
training_args = TrainingArguments(output_dir="./results", per_device_train_batch_size=16, 
                                  evaluation_strategy= "epoch", per_device_eval_batch_size=16, num_train_epochs=5,weight_decay=0.01)

#fine-tuning model
trainer = Trainer(model = model,
              args = training_args,
              train_dataset= hyperpartisan_train,
              eval_dataset= hyperpartisan_valid,
              tokenizer= longformer_tokenizer,
              data_collator=data_collator,
              compute_metrics= compute_metrics)

#searching for best hyperparameters
trainer.train()

In [ ]:
#plot 
fig, ax = optuna.visualization.plot_slice(study, params=["x", "y"])

In [ ]:
#update hyperparameters 
for k,v in best_run.hyperparameters.items():
  setattr(training_args, k, v)

best_trainer = Trainer(model = longformer_model,
                       args = training_args,
                       train_dataset= hyperpartisan_train,
                      eval_dataset= hyperpartisan_eval,
                      tokenizer= longformer_tokenizer,
                      data_collator=data_collator,
                      compute_metrics= compute_metrics)

best_trainer.train()